### Import starting libraries, initial parameters

In [10]:
import matplotlib.pyplot as plt
import random
import numpy as np


random.seed(42)
np.random.seed(42)

### Define initial variables 

In [11]:
# Problem size
nbrblood = 100
nbrcoll = 2000
nbrprivh = 20
nbrpubh = 30


# Define coords for each type of location
x_coordCC = np.random.uniform(0, 100, nbrcoll)
y_coordCC = np.random.uniform(0, 100, nbrcoll)
x_coordBB = np.random.uniform(0, 100, nbrblood)
y_coordBB = np.random.uniform(0, 100, nbrblood)
x_coord_privhosp = np.random.uniform(0, 100, nbrprivh)
y_coord_privhosp = np.random.uniform(0, 100, nbrprivh)
x_coord_pubhosp = np.random.uniform(0, 100, nbrpubh)
y_coord_pubhosp = np.random.uniform(0, 100, nbrpubh)


R = 6371

def deg_to_rad(degrees):
    return degrees*(np.pi/180)

# Use Euclidean distance for random data
distancelct = np.empty([nbrblood, nbrcoll])
for i_index in range(nbrblood):
    for j_index in range(nbrcoll):
        distancelct[i_index, j_index] = np.linalg.norm(
            [x_coordBB[i_index] - x_coordCC[j_index], y_coordBB[i_index] - y_coordCC[j_index]]
        )

distanceBB_priv = np.empty((nbrblood, nbrprivh))
for i_index in range(nbrblood):
    for h_index in range(nbrprivh):
        distanceBB_priv[i_index, h_index] = np.linalg.norm(
            [x_coordBB[i_index] - x_coord_privhosp[h_index], y_coordBB[i_index] - y_coord_privhosp[h_index]]
        )

distanceBB_pub = np.empty((nbrblood, nbrpubh))
for i_index in range(nbrblood):
    for h_index in range(nbrpubh):
        distanceBB_pub[i_index, h_index] = np.linalg.norm(
            [x_coordBB[i_index] - x_coord_pubhosp[h_index], y_coordBB[i_index] - y_coord_pubhosp[h_index]]
        )

# Generate random blood collection amounts and capacities
q_c = np.random.randint(1, 10, nbrcoll)  # Amount of blood collected at each CC
g_b = np.random.randint(50, 100, nbrblood)  # Min blood required at each bank
q_b = np.random.randint(100, 200, nbrblood)  # Max capacity at each bank

# For hospital demand, generate random values for demonstration
hospital_demand = np.random.randint(1, 10, nbrprivh + nbrpubh)

# Prepare coordinates for plotting and TSP
comm_coords = list(zip(x_coordCC, y_coordCC))
bank_coords = list(zip(x_coordBB, y_coordBB))
privhosp_coords = list(zip(x_coord_privhosp, y_coord_privhosp))
pubhosp_coords = list(zip(x_coord_pubhosp, y_coord_pubhosp))

### Initialize ACO Parameters

In [12]:
# Starting parameters
num_ants = 100
num_iterations = 100
alpha = 1
beta = 2
rho = 0.05
Q = 1

# Find total number of objects for ACO
num_objects = nbrblood+nbrcoll+nbrprivh+nbrpubh
objects = [(random.randint(1, 20), random.randint(10, 100)) for _ in range(num_objects)]
capacity = q_b

# Initialize pheromons on each object to be 1
pheromones = np.ones(num_objects)

# calculate value/weight as one indicator for solution quality
heuristic = [val / wt for wt, val in objects]

### ACO iterations

1. Each ant builds a solution
2. Solution evaluated based on value and weight
3. Pheromones updated

In [15]:
# initialization
best_solution = None
best_value = 0
best_values_over_time = []

for iteration in range (num_iterations):
    all_solutions = []
    all_costs = []

    for ant in range(num_ants):
        assignment = []
        bb_blood = np.zeros(nbrblood)
        feasible = True
        total_cost = 0

        for j in range(nbrcoll):  # for each CC
                probs = []
                for i in range(nbrblood):  # for each BB
                    if bb_blood[i] + q_c[j] <= q_b[i]:
                        eta = 1 / (distancelct[i][j] + 1e-6)
                        tau = pheromones[i][j]
                        probs.append((tau ** alpha) * (eta ** beta))
                    else:
                        probs.append(0)

                probs = np.array(probs)
                if probs.sum() == 0:
                    feasible = False
                    break  # this ant cannot complete a valid assignment

                probs /= probs.sum()
                chosen_i = np.random.choice(nbrblood, p=probs)

                assignment.append(chosen_i)
                bb_blood[chosen_i] += q_c[j]
                total_cost += distancelct[chosen_i][j]

        if feasible:
            all_solutions.append(assignment)
            all_costs.append(total_cost)

            if total_cost < best_cost:
                best_cost = total_cost
                best_solution = assignment

    #leave the pheromones on the items selected by the ants.
    pheromones = (1 - rho) * pheromones
    for solution, value in zip(all_solutions, all_costs):
        for j,i in enumerate(solution):
            pheromones[i] += Q / (cost + 1e-6)

    #for printing the best value found so far in each iteration
    best_values_over_time.append(best_value)
    
    print(f"Iteration {iteration}: Best = {best_value}")

IndexError: invalid index to scalar variable.